# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

# **1. Instalación de dependencias**

In [ ]:
!pip install PyPDF2 docx openai pandas numpy
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53893 sha256=f5a094ac9764e5365a1ad6379ac01886958ce1a634d7dbe48a470db9713e7e87
  Stored in directory: /root/.cache/pip/wheels/c1/3e/c3/e81c11effd0be5658a035947c66792dd993bcff317eae0e1ed
Successfully built docx
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [2]:
import gradio as gr
print(gr.__version__)

5.33.1


# **2. Definición de Corpus**

In [3]:
import os
from PyPDF2 import PdfReader

def read_corpus_from_files(root_path: str) -> list[dict[str, str]]:
    corpus: list[dict[str, str]] = []
    for filename in os.listdir(root_path):
        path = os.path.join(root_path, filename)
        if filename.lower().endswith('.pdf'):
          try:
            reader = PdfReader(path)
            text = "\n".join(p.extract_text() or '' for p in reader.pages)
            corpus.append({'title': filename, 'content': text})
          except Exception as e:
            print(f"Error leyendo PDF {filename}: {e}")
        elif filename.lower().endswith('.src'):
            with open(path, 'r', encoding='utf-8') as f:
              corpus.append({'title': filename, 'content': f.read()})
    return corpus

In [5]:
raw_corpus = read_corpus_from_files("corpus_data")
if not raw_corpus:
    raise ValueError("No se encontró archivos .pdf, .docx, .txt o .src en la carpeta indicada")

In [6]:
len(raw_corpus)

1

In [8]:
# Creamos el DataFrame
import pandas as pd
df = pd.DataFrame(raw_corpus)
df

,title,content
0,examples.src,-- Reglas generales:\n-- 0. Todo se basa en la...


# **3. Función de Embedding**

In [9]:
from google.colab import userdata
import numpy as np
import openai

openai.api_key = userdata.get('openapikey')

def embed_text(text: str) -> np.ndarray:
    resp = openai.embeddings.create(
        model="text-embedding-3-large",
        input=text
    )
    return np.array(resp.data[0].embedding)

df['embedding'] = df['content'].apply(embed_text)
df.head()

,title,content,embedding
0,examples.src,-- Reglas generales:\n-- 0. Todo se basa en la...,"[-0.02203100360929966, -1.3361195669858716e-05..."


# **4. Obtención del documento mas relevante**

In [10]:
def find_most_relevant(query: str, df: pd.DataFrame) -> str:
    q_emb = embed_text(query)

    scores = df["embedding"].apply(lambda e: np.dot(e, q_emb))
    idx = int(scores.idxmax())
    return df.loc[idx, "content"]

# **5. Construcción del prompt y llamada LLM**

In [16]:
import textwrap

def make_prompt(question: str, context: str) -> str:
    ctx = context.replace("\n", " ")
    return textwrap.dedent(f"""
        Eres un asistente útil que responde basándose en el siguiente texto de referencia:
        \"{ctx}\"

        Pregunta: \"{question}\"

        Responde de forma clara y completa.
    """)

def make_prompt_no_context(question: str) -> str:
    return textwrap.dedent(f"""
        Eres un asistente útil:
        Pregunta: \"{question}\"
        Responde de forma clara y completa.
    """)

def answer(chat_history, question: str) -> str:
  passage = find_most_relevant(question, df)
  prompt = make_prompt(question, passage)
  resp = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "Eres un asistente experto en programación 1280. Al responder basado en el texto de referencia no hace falta que menciones que traes la data de la referencia. Solo responde la pregunta"},
      {"role": "user", "content": prompt}
    ],
    temperature=0.2
  )
  chat_history = chat_history + [(question, resp.choices[0].message.content.strip())]
  return chat_history, chat_history

def answerWithNoContext(chat_history, question: str) -> str:
  prompt = make_prompt_no_context(question)
  resp = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "Eres un asistente experto."},
      {"role": "user", "content": prompt}
    ],
    temperature=0.2
  )
  chat_history = chat_history + [(question, resp.choices[0].message.content.strip())]
  return chat_history, chat_history

# **6. Interfaz de usuario**

In [17]:
import gradio as gr

with gr.Blocks(title="Asistente de Programación RiceLake 1280") as demo:
    gr.Markdown("## Asistente de Programación RiceLake 1280\nRealiza tu consulta sobre programación 1280")

    txt = gr.Textbox(
        lines=4,
        placeholder="Escribe tu pregunta...",
        label="¿Que quieres saber?"
    )

    with gr.Row():
        with gr.Column():
            btn_rag = gr.Button("Enviar usando RAG")
            out_rag = gr.Chatbot(label="Respuesta con RAG")
            state_rag = gr.State([])
            btn_rag.click(
                fn=answer,
                inputs=[state_rag, txt],
                outputs=[state_rag, out_rag],
            )

        with gr.Column():
            btn_no = gr.Button("Enviar sin usar RAG")
            out_no = gr.Chatbot(label="Respuesta sin RAG")
            state_no = gr.State([])
            btn_no.click(
                fn=answerWithNoContext,
                inputs=[state_no, txt],
                outputs=[state_no, out_no],
            )

demo.launch()

<ipython-input-17-2118794348>:15: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  out_rag = gr.Chatbot(label="Respuesta con RAG")
<ipython-input-17-2118794348>:25: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  out_no = gr.Chatbot(label="Respuesta sin RAG")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a70debd1b9c98b9441.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Conclusiones:**

* #### **Incluyan sus conclusiones de la actividad chatbot LLM + RAG:**



None

# **Fin de la actividad chatbot: LLM + RAG**